In [1]:
# LOW_PRB

"""
PBR : (시가총액) / (총 자본) = (주가 * 총 주식수) / (총 자산 - 총 부채)
"""

'\nPBR : (시가총액) / (총 자본) = (주가 * 총 주식수) / (총 자산 - 총 부채)\n'

In [2]:
# imports

import kquant as kq
import datetime as dt


# TMP

import random
from tqdm import tqdm

In [3]:
# API_account
def set_api_account():
    kq.set_api("KRX2308020", "EQDkUcyI3dK6oIAXqAR8BXOK4bKxHHmH")
    return None

In [4]:
set_api_account()

In [5]:
# LOAD_DATA
# class DATA_LOADER:
class SYMBOL_LOADER:
    @staticmethod
    def load_symbols_df():
        symbols_df = kq.symbol_stock()
        return symbols_df

    class SYMBOL_FILTER:
        @staticmethod
        def filter__market(symbols_df):
            filtered_symbols_df = symbols_df[
                (symbols_df["MARKET"].isin(["코스닥", "유가증권"]))
            ].copy()
            return filtered_symbols_df

        @staticmethod
        def filter__admin_issue(symbols_df):
            filtered_symbols_df = symbols_df[(symbols_df["ADMIN_ISSUE"] == 0)].copy()
            return filtered_symbols_df

        @staticmethod
        def filter_sec_type(symbols_df):
            filtered_symbols_df = symbols_df[
                symbols_df["SEC_TYPE"].isin(["ST", "EF", "EN"])
            ].copy()
            return filtered_symbols_df

    def filter_symbols_df(self, symbols_df):
        symbol_filter = self.SYMBOL_FILTER()
        filtered_symbols_df = symbol_filter.filter__market(symbols_df)
        filtered_symbols_df = symbol_filter.filter__admin_issue(filtered_symbols_df)
        filtered_symbols_df = symbol_filter.filter_sec_type(filtered_symbols_df)
        return filtered_symbols_df

    @staticmethod
    def get_symbols(symbols_df):
        symbols = sorted(set(symbols_df["SYMBOL"]))
        return symbols

    # SYMBOL_LOADER PIPELINE
    def __call__(self):
        symbols_df = self.load_symbols_df()
        filtered_symbols_df = self.filter_symbols_df(symbols_df)
        symbols = self.get_symbols(filtered_symbols_df)
        return symbols

In [6]:
class FUNDAMENTAL_LOADER:
    def __init__(self, symbol, date) -> None:
        self.symbol = symbol
        self.date = date
        self.daily_stock_df = kq.daily_stock(
            symbol,
            start_date=date - dt.timedelta(days=7),
            end_date=date,
        )

    def load_recent_close(self):
        daily_stock_df = self.daily_stock_df
        _close = daily_stock_df.sort_values("DATE").tail(1)["CLOSE"].values[0]
        return _close

    def load_recent_marketcap(self):
        daily_stock_df = self.daily_stock_df
        _marketcap = daily_stock_df.sort_values("DATE").tail(1)["MARKETCAP"].values[0]
        return _marketcap

    def load_recent_netprofit(self):
        netprofit_df = kq.account_history(self.symbol, "122700")
        netprofit_df.sort_values("YEARMONTH", inplace=True)
        _netprofit = netprofit_df.tail(1)["VALUE"].values[0]
        return _netprofit

    def load_recent_capital(self):
        capital_df = kq.account_history(self.symbol, "115000")
        capital_df.sort_values("YEARMONTH", inplace=True)
        _capital = capital_df.tail(1)["VALUE"].values[0]
        return _capital

    def __call__(self):
        _close = self.load_recent_close()
        _marketcap = self.load_recent_marketcap()
        _netprofit = self.load_recent_netprofit()
        _capital = self.load_recent_capital()
        return {
            "symbol": self.symbol,
            "close": _close,
            "marketcap": _marketcap,
            "netprofit": _netprofit,
            "capital": _capital,
        }

In [7]:
symbol_loader = SYMBOL_LOADER()
total_symbols = symbol_loader()

In [8]:
date = dt.date(2023,6,7)

In [9]:
fundamental_data_results = list()
for symbol in tqdm(total_symbols):
    try:
        fundamental_loader = FUNDAMENTAL_LOADER(symbol=symbol, date=date)
        _fundamental_data = fundamental_loader()
        fundamental_data_results.append(_fundamental_data)
    except:
        pass

fundamental_df = pd.DataFrame(fundamental_data_results)

100%|██████████| 3547/3547 [14:46<00:00,  4.00it/s]


In [ ]:
fundamental_df = pd.DataFrame(fundamental_data_results)

In [ ]:
fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]

In [ ]:
fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]
low_pbr_df = fundamental_df.nsmallest(5, "pbr")

In [ ]:
fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]
low_pbr_df = fundamental_df.nsmallest(5, "pbr")
low_pbr_df["pbr_weight"] = low_pbr_df["pbr"].sum() / low_pbr_df["pbr"]

In [ ]:
fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]
low_pbr_df = fundamental_df.nsmallest(5, "pbr")
low_pbr_df["pbr_weight"] = low_pbr_df["pbr"].sum() / low_pbr_df["pbr"]
low_pbr_df["price_invest"] = (
    low_pbr_df["pbr_weight"] / low_pbr_df["pbr_weight"].sum()
) * 10000000

In [ ]:
fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]
low_pbr_df = fundamental_df.nsmallest(5, "pbr")
low_pbr_df["pbr_weight"] = low_pbr_df["pbr"].sum() / low_pbr_df["pbr"]
low_pbr_df["price_invest"] = (
    low_pbr_df["pbr_weight"] / low_pbr_df["pbr_weight"].sum()
) * 10000000
low_pbr_df["cnt_invest"] = low_pbr_df["price_invest"] // low_pbr_df["close"]

In [ ]:
fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]
low_pbr_df = fundamental_df.nsmallest(5, "pbr")
low_pbr_df["pbr_weight"] = low_pbr_df["pbr"].sum() / low_pbr_df["pbr"]
low_pbr_df["price_invest"] = (
    low_pbr_df["pbr_weight"] / low_pbr_df["pbr_weight"].sum()
) * 10000000
low_pbr_df["cnt_invest"] = low_pbr_df["price_invest"] // low_pbr_df["close"]
list(low_pbr_df.set_index("symbol")['cnt_invest'].astype(int).to_dict().items())

[('036530', 240),
 ('128820', 469),
 ('127710', 935),
 ('001290', 2227),
 ('005380', 8)]

In [ ]:
# Signiture
import pandas as pd
import datetime as dt
import logging


def trade_func(
    date: dt.date,
    dict_df_result: dict[str, pd.DataFrame],
    dict_df_position: dict[str, pd.DataFrame],
    logger: logging.Logger,
) -> list[tuple[str, int]]:
    daily_invest_money = 20000000

    """LOAD DATA"""

    """ LOAD DATA (SYMBOL) """
    symbol_loader = SYMBOL_LOADER()
    total_symbols = symbol_loader()

    # tmp
    total_symbols = random.sample(total_symbols, 50)

    """ LOAD DATA (FUNDAMENTAL) """
    fundamental_data_results = list()

    for symbol in total_symbols:
        try:
            fundamental_loader = FUNDAMENTAL_LOADER(symbol=symbol, date=date)
            _fundamental_data = fundamental_loader()
            fundamental_data_results.append(_fundamental_data)
        except:
            pass

    fundamental_df = pd.DataFrame(fundamental_data_results)
    fundamental_df["pbr"] = fundamental_df["marketcap"] / fundamental_df["capital"]
    low_pbr_df = fundamental_df.nsmallest(5, "pbr")
    low_pbr_df["pbr_weight"] = low_pbr_df["pbr"].sum() / low_pbr_df["pbr"]
    low_pbr_df["price_invest"] = (
        low_pbr_df["pbr_weight"] / low_pbr_df["pbr_weight"].sum()
    ) * daily_invest_money
    low_pbr_df["cnt_invest"] = low_pbr_df["price_invest"] // low_pbr_df["close"]
    symbols_and_orders = list(
        low_pbr_df.set_index("symbol")["cnt_invest"].astype(int).to_dict().items()
    )
    print(date)
    return symbols_and_orders

In [ ]:
dict_df_result = kq.backtest_stock_port_daily(
    trade_func,
    "2023-08-01",  # 실제 심사에서는 투자기간 시작일
    "2023-08-30",  # 실제 심사에서는 투자기간 종료일
    init_cash=1_000_000_000,  # 10억원
)

[2023-08-01] 종목: 001040, 주문전 보유수량:      0 주문수량:    132, 매매수량:    132, 주문후 보유수량:    132
[2023-08-01] 종목: 004150, 주문전 보유수량:      0 주문수량:  1,582, 매매수량:  1,582, 주문후 보유수량:  1,582
[2023-08-01] 종목: 005430, 주문전 보유수량:      0 주문수량:     73, 매매수량:     73, 주문후 보유수량:     73
[2023-08-01] 종목: 010400, 주문전 보유수량:      0 주문수량:    398, 매매수량:    398, 주문후 보유수량:    398


2023-08-01


[2023-08-01] 종목: 103140, 주문전 보유수량:      0 주문수량:     46, 매매수량:     46, 주문후 보유수량:     46
[2023-08-02] 종목: 037400, 주문전 보유수량:      0 주문수량:  3,933, 매매수량:  3,933, 주문후 보유수량:  3,933
[2023-08-02] 종목: 084690, 주문전 보유수량:      0 주문수량:    645, 매매수량:    645, 주문후 보유수량:    645
[2023-08-02] 종목: 006200, 주문전 보유수량:      0 주문수량:  3,707, 매매수량:  3,707, 주문후 보유수량:  3,707
[2023-08-02] 종목: 069960, 주문전 보유수량:      0 주문수량:     57, 매매수량:     57, 주문후 보유수량:     57


2023-08-02


[2023-08-02] 종목: 057050, 주문전 보유수량:      0 주문수량:     61, 매매수량:     61, 주문후 보유수량:     61
[2023-08-03] 종목: 016610, 주문전 보유수량:      0 주문수량:  1,491, 매매수량:  1,491, 주문후 보유수량:  1,491
[2023-08-03] 종목: 002920, 주문전 보유수량:      0 주문수량:  1,588, 매매수량:  1,588, 주문후 보유수량:  1,588
[2023-08-03] 종목: 015760, 주문전 보유수량:      0 주문수량:    173, 매매수량:    173, 주문후 보유수량:    173
[2023-08-03] 종목: 104480, 주문전 보유수량:      0 주문수량:  1,470, 매매수량:  1,470, 주문후 보유수량:  1,470


...

[2023-08-28] 종목: 042420, 주문전 보유수량:      0 주문수량:    127, 매매수량:    127, 주문후 보유수량:    127
[2023-08-29] 종목: 002880, 주문전 보유수량:  6,291 주문수량:  9,897, 매매수량:  9,897, 주문후 보유수량: 16,188
[2023-08-29] 종목: 038540, 주문전 보유수량:      0 주문수량:  1,025, 매매수량:  1,025, 주문후 보유수량:  1,025
[2023-08-29] 종목: 084010, 주문전 보유수량:      0 주문수량:    273, 매매수량:    273, 주문후 보유수량:    273
[2023-08-29] 종목: 036420, 주문전 보유수량:      0 주문수량:    214, 매매수량:    214, 주문후 보유수량:    214


2023-08-29


[2023-08-29] 종목: 054940, 주문전 보유수량:      0 주문수량:  2,790, 매매수량:  2,790, 주문후 보유수량:  2,790
[2023-08-30] 종목: 003240, 주문전 보유수량:     11 주문수량:      9, 매매수량:      9, 주문후 보유수량:     20
[2023-08-30] 종목: 001940, 주문전 보유수량:      0 주문수량:    266, 매매수량:    266, 주문후 보유수량:    266
[2023-08-30] 종목: 011370, 주문전 보유수량:  3,518 주문수량:  3,544, 매매수량:  3,544, 주문후 보유수량:  7,062


2023-08-30


[2023-08-30] 종목: 035760, 주문전 보유수량:      0 주문수량:     52, 매매수량:     52, 주문후 보유수량:     52
[2023-08-30] 종목: 053210, 주문전 보유수량:      0 주문수량:    414, 매매수량:    414, 주문후 보유수량:    414


In [ ]:
total_df=dict_df_result['TOTAL']

In [ ]:
total_df

,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-08-01,TOTAL,0,0,0,0,"2,231",0.0000,0,0,0,"-19,974,250","980,025,750","19,974,250","19,974,250","1,000,000,000",0,0,0,"1,000,000,000",0
1,2023-08-02,TOTAL,0,0,0,0,"10,634",0.0000,0,0,0,"-19,977,351","960,048,399","39,951,601","39,853,321","999,901,720",0,"-98,280","-98,280","1,000,000,000","98,280"
2,2023-08-03,TOTAL,0,0,0,0,"15,440",0.0000,0,0,0,"-19,951,365","940,097,034","59,902,966","59,611,092","999,708,126",0,"-291,874","-291,874","1,000,000,000","291,874"
3,2023-08-04,TOTAL,0,0,0,0,"26,515",0.0000,0,0,0,"-19,949,765","920,147,269","79,852,731","80,157,800","1,000,305,069",0,"305,069","305,069","1,000,305,069",0
4,2023-08-07,TOTAL,0,0,0,0,"34,513",0.0000,0,0,0,"-19,947,488","900,199,781","99,800,219","100,476,868","1,000,676,649",0,"676,649","676,649","1,000,676,649",0
5,2023-08-08,TOTAL,0,0,0,0,"40,549",0.0000,0,0,0,"-19,983,062","880,216,719","119,783,281","119,539,877","999,756,596",0,"-243,404","-243,404","1,000,676,649","920,053"
6,2023-08-09,TOTAL,0,0,0,0,"47,580",0.0000,0,0,0,"-19,991,702","860,225,017","139,774,983","140,327,216","1,000,552,233",0,"552,233","552,233","1,000,676,649","124,416"
7,2023-08-10,TOTAL,0,0,0,0,"53,422",0.0000,0,0,0,"-19,987,620","840,237,397","159,762,603","163,019,763","1,003,257,160",0,"3,257,160","3,257,160","1,003,257,160",0
8,2023-08-11,TOTAL,0,0,0,0,"62,304",0.0000,0,0,0,"-19,986,466","820,250,931","179,749,069","184,238,977","1,004,489,908",0,"4,489,908","4,489,908","1,004,489,908",0
9,2023-08-14,TOTAL,0,0,0,0,"66,695",0.0000,0,0,0,"-19,988,070","800,262,861","199,737,139","202,918,918","1,003,181,779",0,"3,181,779","3,181,779","1,004,489,908","1,308,129"
